# Chapter 14: Deploying TensorFlow Models

Notebook ini merupakan hasil reproduksi dan penjelasan teori dari **Bab 14 - Deploying TensorFlow Models** dari buku *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow (2nd Edition)* oleh Aurélien Géron.

📌 Bab ini membahas bagaimana cara menyimpan, memuat, dan melakukan deployment model TensorFlow agar bisa digunakan dalam aplikasi nyata.

---


## Ringkasan Teori Bab 14: Deploying TensorFlow Models

### 1. Menyimpan dan Memuat Model

**Format Keras SavedModel** (default, fleksibel):
```python
model.save("model_name")  # Simpan
model = keras.models.load_model("model_name")  # Load
```

**Format HDF5** (kompatibel lintas platform):
```python
model.save("model_name.h5")
model = keras.models.load_model("model_name.h5")
```

---

### 2. Menyimpan hanya bobot model (weights)

```python
model.save_weights("weights.h5")
model.load_weights("weights.h5")
```

Gunakan jika hanya ingin menyimpan parameter, bukan arsitektur.

---

### 3. SavedModel Format

- Direkomendasikan untuk production dan TensorFlow Serving
- Menyimpan graph, variable, dan metadata
- Bisa digunakan di TensorFlow Lite, TensorFlow.js, atau TensorFlow Serving

Struktur:
```
saved_model/
├── assets/
├── variables/
└── saved_model.pb
```

---

### 4. TensorFlow Serving

Cara mendistribusikan model di server production menggunakan REST atau gRPC API.

- Kompatibel dengan Docker
- Load model dari path direktori
- Kirim permintaan inferensi via HTTP atau gRPC

---

### 5. TensorFlow Lite

Digunakan untuk menjalankan model ML di perangkat mobile (Android, iOS) dan embedded system.

```bash
# Convert model
converter = tf.lite.TFLiteConverter.from_saved_model('model_name')
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
```

---

### 6. TensorFlow.js

Untuk menjalankan model ML di browser (JavaScript)

```bash
tensorflowjs_converter --input_format=tf_saved_model ./model ./tfjs_model
```

---

### 7. Signatures dan Serving Functions

Saat menyimpan model, kita bisa tentukan fungsi serving khusus:
```python
@tf.function(input_signature=[tf.TensorSpec([None, 4], tf.float32)])
def serving_fn(inputs):
    return {'outputs': model(inputs)}
```

---

### 8. Model Versioning dan Deployment Best Practice

- Gunakan versioned directory `/models/my_model/1/`
- Gunakan REST API untuk inference
- Monitoring dan logging penting dalam production
- Jangan lupa normalisasi/preprocessing harus konsisten

---

📌 Kesimpulan:
Dengan memahami format penyimpanan dan alat deployment seperti TF Serving, TF Lite, dan TF.js, kamu bisa membawa model ke dalam aplikasi nyata di web, mobile, atau cloud 🚀

---


## Implementasi Penyimpanan dan Konversi Model TensorFlow

Di sini kita akan:
1. Membangun model sederhana
2. Menyimpan dalam format `.h5` dan `SavedModel`
3. Mengonversi ke format **TFLite**

---


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Dataset dummy
X = np.random.rand(100, 4)
y = np.random.randint(0, 3, size=100)
y_cat = keras.utils.to_categorical(y, num_classes=3)

# Model sederhana
model = keras.models.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(4,)),
    keras.layers.Dense(3, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y_cat, epochs=5, verbose=0)

### 🔒 Menyimpan model dalam format `.h5`
Format ini menyimpan arsitektur, bobot, dan konfigurasi training.


In [ ]:
# Simpan model dalam format HDF5
h5_path = "/mnt/data/model_h5_example.h5"
model.save(h5_path)

# Load kembali model
loaded_model_h5 = keras.models.load_model(h5_path)
print("Model .h5 loaded successfully.")

### 📁 Menyimpan model sebagai `SavedModel` (direktori)
Ini adalah format standar TensorFlow untuk deployment (TF Serving, TF Lite, TF.js).


In [ ]:
# Simpan dalam format SavedModel
savedmodel_path = "/mnt/data/saved_model_example"
model.save(savedmodel_path)

# Load kembali model
loaded_saved_model = keras.models.load_model(savedmodel_path)
print("SavedModel loaded successfully.")

### 📱 Konversi ke TFLite
Model TFLite digunakan untuk menjalankan inference di perangkat mobile atau embedded.


In [ ]:
# Konversi ke TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(savedmodel_path)
tflite_model = converter.convert()

# Simpan ke file
tflite_path = "/mnt/data/model_example.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print("TFLite model created and saved.")

### Penjelasan:
- **`.h5`**: Format praktis dan lintas platform (bisa di-load oleh tools lain seperti MATLAB)
- **`SavedModel`**: Ideal untuk production (TensorFlow Serving, TF.js, TFLite)
- **`.tflite`**: Format ringan untuk deployment di smartphone atau microcontroller

Kamu bisa menguji `.tflite` dengan **interpreter TFLite** untuk menjalankan inference di Android/iOS.

---
